In [10]:
from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource, TapTool, CustomJS, Plot, Quad, HoverTool, Text, Range1d, PanTool
from bokeh.plotting import figure
from bokeh.sampledata.sample_geojson import geojson
from bokeh.tile_providers import get_provider, Vendors
from bokeh.layouts import row, column, widgetbox
from bokeh.models.widgets import PreText, Select

In [2]:
import pandas as pd
import numpy as np

In [27]:
output_file('2019 11 04 w7 visualisation.html')

Считываем таблицы с данными, предподготовленными в отдельном ноутбуке.

In [3]:
data=pd.read_csv('2019 11 01 data w7.csv', header=0)

In [5]:
data_zones=pd.read_csv('2019 11 01 data_zones w7.csv', header=0)

In [7]:
data_zones['d_h']=pd.to_datetime(data_zones['d_h'])#Переводим параметр времени в дайт-тайм формат

In [8]:
data_zones['d_h'][:3]

0   2016-06-01 00:00:00
1   2016-06-01 01:00:00
2   2016-06-01 02:00:00
Name: d_h, dtype: datetime64[ns]

In [9]:
len(data_zones)

714

In [12]:
data.head(2)

,zone,y_1_0,y_1_1,y_1_2,y_1_3,y_1_4,y_1_5,y_1_6,y_1_7,y_1_8,...,30_17_1,30_17_2,30_17_3,30_17_4,30_17_5,30_17_6,w,e,n,s
0,1075,26.0,14.0,5.0,2.0,1.0,7.0,23.0,34.0,72.0,...,107.26300,102.71216,103.74312,114.91366,110.85466,97.88498,-8.240119e+06,-8.238882e+06,4.969174e+06,4.967947e+06
1,1076,30.0,21.0,25.0,5.0,8.0,31.0,54.0,151.0,185.0,...,178.34714,185.65646,154.21811,187.10762,162.30356,129.23576,-8.240119e+06,-8.238882e+06,4.970402e+06,4.969174e+06


In [16]:
data_zones[['y2168', 'p2168']].tail(2)

,y2168,p2168
712,0.0,14.152299
713,0.0,22.001620


#### Попробуем сделать визуализацию.

In [33]:
pre = PreText(text="""Специализация "Машинное обучение и анализ данных".

Финальный проект - "Анализ временных рядов на карте".

Задание - Спрогнозировать число поездок в час из различных зон Нью Йорка в определенный день и определенный час.

Цель - аучиться предсказывать количество поездок в ближайшие часы в каждом районе Нью-Йорка. Для обучения использовались данные
с января 2016г. по май 2016г. Прогнозы составлены на июнь 2016г. Метрика качества - среднее абсолютное отклонение. 
Полученная ошибка - 21.5

Описание разделов проекта:
1. Знакомство с данными и агрегация. Данные были очищены от ошибок и аномалий. Посчитано среднее количество поездок в час из
каждой зоны.
2. Работа с геоданными. Были нарисованы несколько карт с отражением данных по статситкие поездок из каждой зоны. Отобраны 
зоны со средним количеством поездок в час более 5.
3. Прогнозирование рядов со сложной сезонностью. По данным одного из регионов была построена модель SARIMAX.
4. Прогнозирование большого количества рядов. Проведена кластеризация стандартизированных временных рядов на 8 кластеров.
Для каждого кластера построена модель SARIMAX. Сделаны прогнозы на июнь 2016г. для всех отобранных зон. Ошибка составила 46.
5. Прогнозирование с помощью регрессии. Был сформирован сет данных на основе сырых данных. Проведен подбор парамтров при 
помощи обучения по сетке. Расматривались модели Lasso, SGDRegressor, XGBRegressor. Итоге остановился на XGBRegressor. Сделан
прогноз на июнь 2016г. Ошибка составила 22.52
6. Дополнительные признаки. Сформированы дополнительные признаки. Проведен подбор гиперпараметров на данных за май. Сделан 
прогноз поездок на июнь 2016г. Ошибка составила 21.5. Были добавлены географические, временные и признаке на основе 
количества поездок за предыдущие дни. Так же добавлен признак времени года.
7. Оформление проекта. На базе пакета BOKEH сделана визуализация результатов работы модели. На карте изображен Нью Йорк и 
сетка зон, со средним числом поездок более 5 в час. При выборе зоны выводится информация о номере зоны, фактическом значении 
числа поездок в час и предсказания на 1 - 6 часов вперед. Можно выбрать день в июне и час в дне для которых будет выводиться 
информация. Приведен график для выбранного дня предсказанных, при предсказывании на 1 час вперед, количеств поездок в час и 
фактические количества поездок в это предсказанный час. Приведен график на месяц предсказанных количеств поездок, при 
предсказании на 6 часов вперед, и фактические количества поездок в это предсказанный час.

Используя выбор региона на карте и задавая день и час можно посмотреть значения фактического количества поездок на этот час и
предсказания с 1 по 6 часов вперед."""
)

In [34]:
#Создаем источники данных
source = ColumnDataSource(data)#Для общих нужд
plot_source=ColumnDataSource(data=dict(r=list(data['zone']), #Для карты и регионов на ней
                                       left=list(data['w']),
                                       right=list(data['e']),
                                       top=list(data['n']),
                                       bottom=list(data['s']),
                                       fv=data['y_1_0'],
                                       pv_1=data['1_0_1'],
                                       pv_2=data['1_0_2'],
                                       pv_3=data['1_0_3'],
                                       pv_4=data['1_0_4'],
                                       pv_5=data['1_0_5'],
                                       pv_6=data['1_0_6']
                                       ))

graf_source=ColumnDataSource(data=dict(x=range(17), #Для рисования графиков для выбранного дня
                                      yfact_1=list(data[['y_1_1', 'y_1_2', 'y_1_3', 'y_1_4','y_1_5', 'y_1_6', 'y_1_7', 'y_1_8',
                                                        'y_1_9', 'y_1_10', 'y_1_11', 'y_1_12', 'y_1_13', 'y_1_14', 'y_1_15',
                                                        'y_1_16', 'y_1_17']].loc[0]),#Фактическ значения через час 
                                      pred_1=list(data[['1_0_1', '1_1_1', '1_2_1', '1_3_1', '1_4_1', '1_5_1', '1_6_1', '1_7_1',
                                                        '1_8_1', '1_9_1', '1_10_1', '1_11_1', '1_12_1', '1_13_1', '1_14_1',
                                                        '1_15_1', '1_16_1']].loc[0]),#Предсказания на час вперед 
                                      ))#Это параметр - индекс выбранной зоны

series_source=ColumnDataSource(data=data_zones) #Для графиков рядов для целого месяца.
series_source.data['y_f']=series_source.data['y1075']#Добавляем парам - фактич-е значение в предсказанный час для выбран зоны
series_source.data['y_p']=series_source.data['p1075']#Добавляем парам - предсказ-е значение в предсказанный час для выбран зоны

text_source=ColumnDataSource(data=dict(r=['1075'],
                                       f=[str(data['y_1_0'][0])],
                                       p_1=[str(data['1_0_1'][0])],
                                       p_2=[str(data['1_0_2'][0])],
                                       p_3=[str(data['1_0_3'][0])],
                                       p_4=[str(data['1_0_4'][0])],
                                       p_5=[str(data['1_0_5'][0])],
                                       p_6=[str(data['1_0_6'][0])]))

add_source=ColumnDataSource(data=dict(hsv=[0, 1],
                                       dsv=[1, 1],
                                       zone_in=[0, 1]))#Это параметр - индекс выбранной зоны
#Создаем глиф рисования карты и регионов
map_box = figure(tools = 'tap')
tile_provider = get_provider(Vendors.CARTODBPOSITRON_RETINA)#Создаем рисунок карты                                                                               
map_box.add_tile(tile_provider)#Добавляем рисунок карты в глиф
#Создаем квадраты зон на карте
quad=Quad(top='top', bottom='bottom', left='left',
          right='right', fill_alpha=0.2, fill_color="navy", line_alpha=0.7, line_color="navy")
map_box.add_glyph(plot_source, quad)#Добавляем зоны в глиф
map_box.add_tools(HoverTool(tooltips="""Зона @r<br />Поездки из зоны в выбранный час @fv<br />
                                        Прогнозируемое число поездок через 1 час @pv_1<br />
                                        Прогнозируемое число поездок через 2 часа @pv_2<br />
                                        Прогнозируемое число поездок через 3 часа @pv_3<br />
                                        Прогнозируемое число поездок через 4 часа @pv_4<br />
                                        Прогнозируемое число поездок через 5 часов @pv_5<br />
                                        Прогнозируемое число поездок через 6 часов @pv_6<br />"""))#Добавляем хувер
map_box.add_tools(PanTool())#Добавляем передвигатель карты

#Создаем графики фактических значений и предсказаний на 1 час вперед для выбранного дня                             
graf_figure=figure(plot_width=800, plot_height=400, y_axis_label="Кол-во поездок", x_axis_label="Часы", 
                   title='Графики предсказанного на час вперед числа поездок и фактического числа поездок на это час')
fact_line=graf_figure.line(x='x', y='yfact_1', color='red', source=graf_source, legend='Факт')
pred_line=graf_figure.line(x='x', y='pred_1', color='green', source=graf_source, legend='Предсказание')

#Создаем графики фактических значений и предсказаний на 6 часов вперед для выбранного региона за весь месяц                            
series_figure=figure(plot_width=1400, plot_height=400, y_axis_label="Кол-во поездок", x_axis_label="Даты_Часы", 
                   title='Графики предсказанного на 6 часов вперед числа поездок и фактического числа поездок в этот час',
                    x_axis_type="datetime")
fact_series=series_figure.line(x='d_h', y='y_f', color='red', source=series_source, legend='Факт')
pred_series=series_figure.line(x='d_h', y='y_p', color='green', source=series_source, legend='Предсказание')

#Создаем глифы для вывода текста со значениями числа поездок и предсказаний для выбранного дня и часа
text_r=figure(plot_width=400, plot_height=80, tools='',
                title='Выбранная Зона')
text_f=figure(plot_width=400, plot_height=80, tools='',
                title='Поездок из выбранного региона в выбранный час')
text_p_1=figure(plot_width=400, plot_height=80, tools='',
                title='Прогноз поездок из выбранного региона через 1 час')
text_p_2=figure(plot_width=400, plot_height=80, tools='',
                title='Прогноз поездок из выбранного региона через 2 часа')
text_p_3=figure(plot_width=400, plot_height=80, tools='',
                title='Прогноз поездок из выбранного региона через 3 часа')
text_p_4=figure(plot_width=400, plot_height=80, tools='',
                title='Прогноз поездок из выбранного региона через 4 часа')
text_p_5=figure(plot_width=400, plot_height=80, tools='',
                title='Прогноз поездок из выбранного региона через 5 часов')
text_p_6=figure(plot_width=400, plot_height=80, tools='',
                title='Прогноз поездок из выбранного региона через 6 час')
glyph_tr=Text(text='r')
glyph_tf=Text(text='f')
glyph_tp_1=Text(text='p_1')
glyph_tp_2=Text(text='p_2')
glyph_tp_3=Text(text='p_3')
glyph_tp_4=Text(text='p_4')
glyph_tp_5=Text(text='p_5')
glyph_tp_6=Text(text='p_6')
text_r.add_glyph(text_source, glyph_tr)
text_f.add_glyph(text_source, glyph_tf)
text_p_1.add_glyph(text_source, glyph_tp_1)
text_p_2.add_glyph(text_source, glyph_tp_2)
text_p_3.add_glyph(text_source, glyph_tp_3)
text_p_4.add_glyph(text_source, glyph_tp_4)
text_p_5.add_glyph(text_source, glyph_tp_5)
text_p_6.add_glyph(text_source, glyph_tp_6)
text_r.axis.visible = False
text_f.axis.visible = False
text_p_1.axis.visible = False
text_p_2.axis.visible = False
text_p_3.axis.visible = False
text_p_4.axis.visible = False
text_p_5.axis.visible = False
text_p_6.axis.visible = False
text_r.grid.visible = False
text_f.grid.visible = False
text_p_1.grid.visible = False
text_p_2.grid.visible = False
text_p_3.grid.visible = False
text_p_4.grid.visible = False
text_p_5.grid.visible = False
text_p_6.grid.visible = False
text_r.outline_line_color = None
text_f.outline_line_color = None
text_p_1.outline_line_color = None
text_p_2.outline_line_color = None
text_p_3.outline_line_color = None
text_p_4.outline_line_color = None
text_p_5.outline_line_color = None
text_p_6.outline_line_color = None

# Создаем калбек для выбора дня
def callback_d (source=source, plot_source=plot_source, graf_source=graf_source, text_source=text_source, 
                add_source=add_source):
    #Изменения в источнике для карты
    dat_a=add_source.data
    dat_s=source.data
    dat_p=plot_source.data
    f=cb_obj.value
    dat_p['fv']=dat_s['y_'+f+'_'+dat_a['hsv'][0]]
    dat_p['pv_1']=dat_s[f+'_'+dat_a['hsv'][0]+'_1']
    dat_p['pv_2']=dat_s[f+'_'+dat_a['hsv'][0]+'_2']
    dat_p['pv_3']=dat_s[f+'_'+dat_a['hsv'][0]+'_3']
    dat_p['pv_4']=dat_s[f+'_'+dat_a['hsv'][0]+'_4']
    dat_p['pv_5']=dat_s[f+'_'+dat_a['hsv'][0]+'_5']
    dat_p['pv_6']=dat_s[f+'_'+dat_a['hsv'][0]+'_6']
    dat_a['dsv']=[f, 1]
    plot_source.change.emit()
    add_source.change.emit()
    #Изменения в источнике для графиков на день
    graf_s=graf_source.data
    graf_s['yfact_1']=[dat_s['y_'+f+'_1'][dat_a['zone_in'][0]], dat_s['y_'+f+'_2'][dat_a['zone_in'][0]],
                       dat_s['y_'+f+'_3'][dat_a['zone_in'][0]], dat_s['y_'+f+'_4'][dat_a['zone_in'][0]], 
                       dat_s['y_'+f+'_5'][dat_a['zone_in'][0]], dat_s['y_'+f+'_6'][dat_a['zone_in'][0]],
                       dat_s['y_'+f+'_7'][dat_a['zone_in'][0]], dat_s['y_'+f+'_8'][dat_a['zone_in'][0]], 
                       dat_s['y_'+f+'_9'][dat_a['zone_in'][0]], dat_s['y_'+f+'_10'][dat_a['zone_in'][0]], 
                       dat_s['y_'+f+'_11'][dat_a['zone_in'][0]], dat_s['y_'+f+'_12'][dat_a['zone_in'][0]],
                       dat_s['y_'+f+'_13'][dat_a['zone_in'][0]], dat_s['y_'+f+'_14'][dat_a['zone_in'][0]], 
                       dat_s['y_'+f+'_15'][dat_a['zone_in'][0]], dat_s['y_'+f+'_16'][dat_a['zone_in'][0]], 
                       dat_s['y_'+f+'_17'][dat_a['zone_in'][0]]]
    graf_s['pred_1']=[dat_s[f+'_0_1'][dat_a['zone_in'][0]], dat_s[f+'_1_1'][dat_a['zone_in'][0]], 
                      dat_s[f+'_2_1'][dat_a['zone_in'][0]], dat_s[f+'_3_1'][dat_a['zone_in'][0]], 
                      dat_s[f+'_4_1'][dat_a['zone_in'][0]], dat_s[f+'_5_1'][dat_a['zone_in'][0]],
                      dat_s[f+'_6_1'][dat_a['zone_in'][0]], dat_s[f+'_7_1'][dat_a['zone_in'][0]], 
                      dat_s[f+'_8_1'][dat_a['zone_in'][0]], dat_s[f+'_9_1'][dat_a['zone_in'][0]], 
                      dat_s[f+'_10_1'][dat_a['zone_in'][0]], dat_s[f+'_11_1'][dat_a['zone_in'][0]],
                      dat_s[f+'_12_1'][dat_a['zone_in'][0]], dat_s[f+'_13_1'][dat_a['zone_in'][0]], 
                      dat_s[f+'_14_1'][dat_a['zone_in'][0]], dat_s[f+'_15_1'][dat_a['zone_in'][0]], 
                      dat_s[f+'_16_1'][dat_a['zone_in'][0]]]
    graf_source.change.emit()
    
    #Изменения в источнике для текстов
    dat_t=text_source.data
    dat_t['f']=[str(dat_s['y_'+f+'_'+dat_a['hsv'][0]][dat_a['zone_in'][0]])]
    dat_t['p_1']=[str(dat_s[f+'_'+dat_a['hsv'][0]+'_1'][dat_a['zone_in'][0]])]
    dat_t['p_2']=[str(dat_s[f+'_'+dat_a['hsv'][0]+'_2'][dat_a['zone_in'][0]])]
    dat_t['p_3']=[str(dat_s[f+'_'+dat_a['hsv'][0]+'_3'][dat_a['zone_in'][0]])]
    dat_t['p_4']=[str(dat_s[f+'_'+dat_a['hsv'][0]+'_4'][dat_a['zone_in'][0]])]
    dat_t['p_5']=[str(dat_s[f+'_'+dat_a['hsv'][0]+'_5'][dat_a['zone_in'][0]])]
    dat_t['p_6']=[str(dat_s[f+'_'+dat_a['hsv'][0]+'_6'][dat_a['zone_in'][0]])]
    text_source.change.emit()

# Создаем калбек для выбора часа    
def callback_h (source=source, plot_source=plot_source, text_source=text_source, add_source=add_source):
    #Изменение источника для карты
    dat_a=add_source.data
    dat_s=source.data
    dat_p=plot_source.data
    f=cb_obj.value
    dat_p['fv']=dat_s['y_'+dat_a['dsv'][0]+'_'+f]
    dat_p['pv_1']=dat_s[dat_a['dsv'][0]+'_'+f+'_1']
    dat_p['pv_2']=dat_s[dat_a['dsv'][0]+'_'+f+'_2']
    dat_p['pv_3']=dat_s[dat_a['dsv'][0]+'_'+f+'_3']
    dat_p['pv_4']=dat_s[dat_a['dsv'][0]+'_'+f+'_4']
    dat_p['pv_5']=dat_s[dat_a['dsv'][0]+'_'+f+'_5']
    dat_p['pv_6']=dat_s[dat_a['dsv'][0]+'_'+f+'_6']
    dat_a['hsv']=[f, 1]
    plot_source.change.emit()
    add_source.change.emit()
    
    #Изменение источника для текстов
    dat_t=text_source.data
    dat_t['f']=[str(dat_s['y_'+dat_a['dsv'][0]+'_'+f][dat_a['zone_in'][0]])]
    dat_t['p_1']=[str(dat_s[dat_a['dsv'][0]+'_'+f+'_1'][dat_a['zone_in'][0]])]
    dat_t['p_2']=[str(dat_s[dat_a['dsv'][0]+'_'+f+'_2'][dat_a['zone_in'][0]])]
    dat_t['p_3']=[str(dat_s[dat_a['dsv'][0]+'_'+f+'_3'][dat_a['zone_in'][0]])]
    dat_t['p_4']=[str(dat_s[dat_a['dsv'][0]+'_'+f+'_4'][dat_a['zone_in'][0]])]
    dat_t['p_5']=[str(dat_s[dat_a['dsv'][0]+'_'+f+'_5'][dat_a['zone_in'][0]])]
    dat_t['p_6']=[str(dat_s[dat_a['dsv'][0]+'_'+f+'_6'][dat_a['zone_in'][0]])]
    text_source.change.emit()
    
#Создаю калбек для реакции на выбор региона на карте.
def code (text_source=text_source, graf_source=graf_source, source=source, plot_source=plot_source, 
          series_source=series_source, add_source=add_source):
    s_i = cb_data.source.selected.indices[0]
    dat_a=add_source.data
    graf_s=graf_source.data
    dat_s=source.data
    dat_p=plot_source.data
    #Изменения в источнике для графиков на день
    graf_s['yfact_1']=[dat_s['y_'+dat_a['dsv'][0]+'_1'][s_i], dat_s['y_'+dat_a['dsv'][0]+'_2'][s_i],
                       dat_s['y_'+dat_a['dsv'][0]+'_3'][s_i], dat_s['y_'+dat_a['dsv'][0]+'_4'][s_i],
                       dat_s['y_'+dat_a['dsv'][0]+'_5'][s_i], dat_s['y_'+dat_a['dsv'][0]+'_6'][s_i],
                       dat_s['y_'+dat_a['dsv'][0]+'_7'][s_i], dat_s['y_'+dat_a['dsv'][0]+'_8'][s_i],
                       dat_s['y_'+dat_a['dsv'][0]+'_9'][s_i], dat_s['y_'+dat_a['dsv'][0]+'_10'][s_i],
                       dat_s['y_'+dat_a['dsv'][0]+'_11'][s_i], dat_s['y_'+dat_a['dsv'][0]+'_12'][s_i],
                       dat_s['y_'+dat_a['dsv'][0]+'_13'][s_i], dat_s['y_'+dat_a['dsv'][0]+'_14'][s_i], 
                       dat_s['y_'+dat_a['dsv'][0]+'_15'][s_i], dat_s['y_'+dat_a['dsv'][0]+'_16'][s_i],
                       dat_s['y_'+dat_a['dsv'][0]+'_17'][s_i]]
    graf_s['pred_1']=[dat_s[dat_a['dsv'][0]+'_0_1'][s_i], dat_s[dat_a['dsv'][0]+'_1_1'][s_i], 
                      dat_s[dat_a['dsv'][0]+'_2_1'][s_i], dat_s[dat_a['dsv'][0]+'_3_1'][s_i],
                      dat_s[dat_a['dsv'][0]+'_4_1'][s_i], dat_s[dat_a['dsv'][0]+'_5_1'][s_i],
                      dat_s[dat_a['dsv'][0]+'_6_1'][s_i], dat_s[dat_a['dsv'][0]+'_7_1'][s_i], 
                      dat_s[dat_a['dsv'][0]+'_8_1'][s_i], dat_s[dat_a['dsv'][0]+'_9_1'][s_i], 
                      dat_s[dat_a['dsv'][0]+'_10_1'][s_i], dat_s[dat_a['dsv'][0]+'_11_1'][s_i],
                      dat_s[dat_a['dsv'][0]+'_12_1'][s_i], dat_s[dat_a['dsv'][0]+'_13_1'][s_i], 
                      dat_s[dat_a['dsv'][0]+'_14_1'][s_i], dat_s[dat_a['dsv'][0]+'_15_1'][s_i], 
                      dat_s[dat_a['dsv'][0]+'_16_1'][s_i]]
    dat_a['zone_in']=[s_i, 1]
    graf_source.change.emit()
    add_source.change.emit()
    
    #Изменения в источнике для графиков на месяц
    dat_ss=series_source.data
    dat_ss['y_f']=dat_ss['y'+str(dat_p['r'][s_i])]
    dat_ss['y_p']=dat_ss['p'+str(dat_p['r'][s_i])]
    series_source.change.emit()
    
    #Изменения в источнике для текстов
    dat_t=text_source.data
    dat_t['r']=[str(dat_s['zone'][s_i])]
    dat_t['f']=[str(dat_s['y_'+dat_a['dsv'][0]+'_'+dat_a['hsv'][0]][s_i])]
    dat_t['p_1']=[str(dat_s[dat_a['dsv'][0]+'_'+dat_a['hsv'][0]+'_1'][s_i])]
    dat_t['p_2']=[str(dat_s[dat_a['dsv'][0]+'_'+dat_a['hsv'][0]+'_2'][s_i])]
    dat_t['p_3']=[str(dat_s[dat_a['dsv'][0]+'_'+dat_a['hsv'][0]+'_3'][s_i])]
    dat_t['p_4']=[str(dat_s[dat_a['dsv'][0]+'_'+dat_a['hsv'][0]+'_4'][s_i])]
    dat_t['p_5']=[str(dat_s[dat_a['dsv'][0]+'_'+dat_a['hsv'][0]+'_5'][s_i])]
    dat_t['p_6']=[str(dat_s[dat_a['dsv'][0]+'_'+dat_a['hsv'][0]+'_6'][s_i])]
    text_source.change.emit()

#Создаем виджеты выбора дня и часа
day_select = Select(options=[str(i) for i in range(1, 31)], value='1', title='Выберите день в июне 2016г.', 
                    callback=CustomJS.from_py_func(callback_d))
hour_select = Select(options=[str(i) for i in range(18)], value='0', title='Выберите час.',
                    callback=CustomJS.from_py_func(callback_h))
widget_row = row(widgetbox(day_select), widgetbox(hour_select))

taptool = map_box.select(type=TapTool)
taptool.callback = CustomJS.from_py_func(code)

rowa = column(pre, column(row(map_box, column(widget_row, graf_figure, row(text_r, text_f),
                                 row(text_p_1, text_p_2), row(text_p_3, text_p_4),
                                 row(text_p_5, text_p_6))),  series_figure))

                             


In [35]:
show(rowa)